In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sql_adapter import SQLAdapter

In [3]:
driver = '{ODBC Driver 17 for SQL Server}'
database = 'PaymentData'
server = 'localhost,1433'
user = 'sa'
password = 'VeryComplicatedPassowrd1234!'

In [4]:
adapter = SQLAdapter(driver, database, server, user, password)

## **Data Generation**

In [4]:
adapter = SQLAdapter(driver, database, server, user, password)
adapter.clean_tables()

In [6]:
%%time
adapter.generate(n_participants=100, n_projects=100, n_payments=100)

CPU times: user 402 ms, sys: 16.5 ms, total: 419 ms
Wall time: 2.41 s


In [10]:
adapter.clean_tables()

In [11]:
%%time
adapter.generate(n_participants=1000, n_projects=1000, n_payments=1000)

CPU times: user 3.83 s, sys: 225 ms, total: 4.06 s
Wall time: 32.3 s


In [13]:
adapter.clean_tables()

## **Balance test**

In [7]:
def insert_into_payment_participant(adapter, oid, balance, name, object_type, bank_details):
    object_type = adapter.OBJECT_TYPES.index(object_type)
    query = f"""INSERT INTO [dbo].[PaymentParticipant] ([Oid], [Balance], [Name], [OptimisticLockField], [GCRecord], 
       [ObjectType], [ActiveFrom], [InactiveFrom], [BankDetails], [Balance2], [Balance3]) 
       VALUES ('{oid}', {balance}, '{name}', 0, Null, {object_type}, '2000-01-01 00:00:00.000', 
       '2020-01-01 00:00:00.000', '{bank_details}', 0, 0)"""

    adapter.execute(query)
    adapter.commit()

    
def insert_bank(adapter, bank_oid, account_type):
    account_type_oid = adapter.execute(f"SELECT Oid FROM [dbo].[AccountType] WHERE name = N'{account_type}'").fetchval()
    query = f"""INSERT INTO [dbo].[Bank] ([Oid], [AccountType]) VALUES ('{bank_oid}', '{account_type_oid}')"""
    adapter.execute(query)
    adapter.commit()
    
    
def insert_supplier(adapter, supplier_oid, contact, profit_by_material_as_payer, profit_by_material_as_payee, cost_by_material_as_payer):
    query = f"""INSERT INTO [dbo].[Supplier] ([Oid], [Contact], [ProfitByMaterialAsPayer], [ProfitByMaterialAsPayee], [CostByMaterialAsPayer])
                VALUES ('{supplier_oid}', '{contact}', {profit_by_material_as_payer}, {profit_by_material_as_payee}, {cost_by_material_as_payer})"""
    adapter.execute(query)
    adapter.commit()
    

def insert_client(adapter, client_oid, first_name, second_name, phone):
    query = f"""INSERT INTO [dbo].[Client] ([Oid], [FirstName], [SecondName], [Phone])
                VALUES ('{client_oid}', '{first_name}', '{second_name}', '{phone}')"""
    adapter.execute(query)
    adapter.commit()

    
def insert_cashbox(adapter, cashbox_oid, account_type):
    account_type_oid = adapter.execute(f"SELECT Oid FROM [dbo].[AccountType] WHERE name = N'{account_type}'").fetchval()
    query = f"""INSERT INTO [dbo].[Cashbox] ([Oid], [AccountType]) VALUES ('{cashbox_oid}', '{account_type_oid}')"""
    adapter.execute(query)
    adapter.commit()
    
def insert_employee(adapter, oid, second_name, first_name, plan_fix_money_request_task):
    query = f"""INSERT INTO [dbo].[Employee] ([Oid], [BusyUntil], [SecondName], [Stuff], [HourPrice], [Patronymic], 
                [PlanfixId], [Head], [PlanfixMoneyRequestTask])
                VALUES ('{oid}', '2020-01-01 00:00:00.000', '{second_name}', 1, 100, '{first_name}', 
                         1, Null, '{plan_fix_money_request_task}')"""
    adapter.execute(query)
    adapter.commit()


def insert_project(adapter, project_oid, name, client, manager, foreman, balance, balance_by_material, balance_by_work):
    query = f"""INSERT INTO [dbo].[Project] ([Oid], [Name], [Address], [Client], [Manager], [Foreman], [OptimisticLockField], [GCRecord], 
               [Balance], [BalanceByMaterial], [BalanceByWork], [PlaningStartDate], [Status], [FinishDate], [Area], [WorkPriceRate],
               [WorkersPriceRate], [RemainderTheAdvance], [PlanfixWorkTask], [PlanfixChangeRequestTask], [UseAnalytics]) 
               VALUES ('{project_oid}', '{name}', 'Test Address', '{client}', '{manager}', '{foreman}', 1, 
                       Null, {balance}, {balance_by_material}, {balance_by_work}, '2000-01-01 00:00:00.000', 1, '2020-01-01 00:00:00.000', 
                        10, 1000000.00, 1000.00, 0, 'Test Task', 
                       'Test Task', 1)"""
    adapter.execute(query)
    adapter.commit()
    
    
def insert_into_payment(adapter, amount, category, project, payer, payee):
    oid = adapter.execute("SELECT NEWID()").fetchval()
    category = adapter.execute(f"SELECT Oid FROM [dbo].[PaymentCategory] WHERE name = N'{category}'").fetchval()
    query = f"""INSERT INTO [dbo].[Payment] ([Oid], [Amount], [Category], [Project], [Justification], [Comment], [Date], [Payer], [Payee],
               [OptimisticLockField], [GCRecord], [CreateDate], [CheckNumber], [IsAuthorized], [Number]) 
               VALUES ('{oid}', {amount}, '{category}', '{project}', 'Test justificatoon', 'Test comment', '2000-01-01 00:00:00.000', '{payer}', '{payee}', 
               0, Null, '2015-01-01 00:00:00.000', '12345', 1, '12345')"""
    adapter.execute(query)
    adapter.commit()
    

def print_balance(bank_oid, supplier_oid, client_oid, cashbox_oid):
    query = "SELECT Balance FROM [dbo].[PaymentParticipant] WHERE Oid = '{}'"
    print("Current balance:")
    print("Bank:", adapter.execute(query.format(bank_oid)).fetchval())
    print("Supplier:", adapter.execute(query.format(supplier_oid)).fetchval())
    print("Client:", adapter.execute(query.format(client_oid)).fetchval())
    print("Cashbox:", adapter.execute(query.format(cashbox_oid)).fetchval())

In [8]:
def test_balance():
    bank_oid = adapter.execute("SELECT NEWID()").fetchval()
    supplier_oid = adapter.execute("SELECT NEWID()").fetchval()
    client_oid = adapter.execute("SELECT NEWID()").fetchval()
    cashbox_oid = adapter.execute("SELECT NEWID()").fetchval()
    manager_oid = adapter.execute("SELECT NEWID()").fetchval()
    foreman_oid = adapter.execute("SELECT NEWID()").fetchval()
    project_oid = adapter.execute("SELECT NEWID()").fetchval()
    
    insert_into_payment_participant(adapter, oid=bank_oid, balance=0, name="Test Bank", object_type='cash', bank_details='bank details')
    insert_into_payment_participant(adapter, oid=supplier_oid, balance=0, name="Test Supplier", object_type='cash', bank_details=None)
    insert_into_payment_participant(adapter, oid=client_oid, balance=0, name="Test Client", object_type='cash', bank_details=None)
    insert_into_payment_participant(adapter, oid=cashbox_oid, balance=0, name="Test Cashbox", object_type='cash', bank_details=None)
    insert_into_payment_participant(adapter, oid=manager_oid, balance=0, name="Test Manager", object_type='cash', bank_details=None)
    insert_into_payment_participant(adapter, oid=foreman_oid, balance=0, name="Test Foreman", object_type='cash', bank_details=None)
    
    insert_bank(adapter, bank_oid, account_type='Авансовый')
    insert_cashbox(adapter, cashbox_oid=cashbox_oid, account_type='Текущий')
    insert_client(adapter, client_oid=client_oid, first_name='Test First Name', second_name='Test Second Name', phone="88005553535")
    insert_supplier(adapter, supplier_oid=supplier_oid, contact='Test Contact', profit_by_material_as_payer=0, 
                    profit_by_material_as_payee=1, cost_by_material_as_payer=0)
    
    insert_employee(adapter, manager_oid,  second_name='Test Second Name', first_name="Test First Name", plan_fix_money_request_task='Test Task')
    insert_employee(adapter, foreman_oid,  second_name='Test Second Name', first_name="Test First Name", plan_fix_money_request_task='Test Task')
    
    insert_project(adapter, project_oid, "Test Project", client_oid, manager_oid, foreman_oid, balance=0, balance_by_material=0, balance_by_work=0)
    
    query = "SELECT Balance FROM [dbo].[PaymentParticipant] WHERE Oid = '{}'"
    
    # Исходное состояние
    assert adapter.execute(query.format(bank_oid)).fetchval() == 0
    assert adapter.execute(query.format(supplier_oid)).fetchval() == 0
    assert adapter.execute(query.format(client_oid)).fetchval() == 0
    assert adapter.execute(query.format(cashbox_oid)).fetchval() == 0
    print_balance(bank_oid, supplier_oid, client_oid, cashbox_oid)
    print()
    
    # Перевели денег поставщику для будущих закупок. Образовался дол банку.
    insert_into_payment(adapter, amount=400000, category='Авансовые платежи', project=project_oid, payer=bank_oid, payee=supplier_oid)
    assert adapter.execute(query.format(bank_oid)).fetchval() == -400000
    assert adapter.execute(query.format(supplier_oid)).fetchval() == 400000
    assert adapter.execute(query.format(client_oid)).fetchval() == 0
    assert adapter.execute(query.format(cashbox_oid)).fetchval() == 0
    print_balance(bank_oid, supplier_oid, client_oid, cashbox_oid)
    print()
    
    # Для клиента приобрели материал. Поставщик нам должен меньше, а клииент должен за материалы
    insert_into_payment(adapter, amount=100000, category='Закупка материалов', project=project_oid, payer=supplier_oid, payee=client_oid)
    assert adapter.execute(query.format(bank_oid)).fetchval() == -400000
    assert adapter.execute(query.format(supplier_oid)).fetchval() == 300000
    assert adapter.execute(query.format(client_oid)).fetchval() == 100000
    assert adapter.execute(query.format(cashbox_oid)).fetchval() == 0
    print_balance(bank_oid, supplier_oid, client_oid, cashbox_oid)
    print()
    
    # Клиент передал наличные для закупки материалов
    insert_into_payment(adapter, amount=150000, category='Закупка материалов', project=project_oid, payer=client_oid, payee=cashbox_oid)
    assert adapter.execute(query.format(bank_oid)).fetchval() == -400000
    assert adapter.execute(query.format(supplier_oid)).fetchval() == 300000
    assert adapter.execute(query.format(client_oid)).fetchval() == -50000
    assert adapter.execute(query.format(cashbox_oid)).fetchval() == 150000
    print_balance(bank_oid, supplier_oid, client_oid, cashbox_oid)
    print()
    
    # Частично гасим кредит
    insert_into_payment(adapter, amount=100000, category='Возврат кредита', project=project_oid, payer=cashbox_oid, payee=bank_oid)
    assert adapter.execute(query.format(bank_oid)).fetchval() == -300000
    assert adapter.execute(query.format(supplier_oid)).fetchval() == 300000
    assert adapter.execute(query.format(client_oid)).fetchval() == -50000
    assert adapter.execute(query.format(cashbox_oid)).fetchval() == 50000

In [9]:
test_balance()

Current balance:
Bank: 0
Supplier: 0
Client: 0
Cashbox: 0

Current balance:
Bank: -400000
Supplier: 400000
Client: 0
Cashbox: 0

Current balance:
Bank: -400000
Supplier: 300000
Client: 100000
Cashbox: 0

Current balance:
Bank: -400000
Supplier: 300000
Client: -50000
Cashbox: 150000



## **Indexes**

In [10]:
from time import perf_counter

In [11]:
adapter.clean_tables()
adapter.generate(n_participants=1000, n_projects=100, n_payments=0)

### **Оценка времени работы с исходными идексами**

In [12]:
payments = [100, 1000, 10000]
for n in payments:
    adapter.clean_tables(tables=['Payment'])
    start_time = perf_counter()
    adapter.generate_payments(n)
    total_time = perf_counter() - start_time
    print(f'Number of inserted payments: {n}, total time: {total_time} seconds')

Number of inserted payments: 100, total time: 0.7553450190025615 seconds
Number of inserted payments: 1000, total time: 8.14556324199657 seconds
Number of inserted payments: 10000, total time: 750.734507909001 seconds


### **Добавляем новые индексы**

In [13]:
create_index_queries = [
"CREATE NONCLUSTERED INDEX iName_AccountType ON [dbo].[AccountType] (Name)",
"CREATE NONCLUSTERED INDEX iProfitByMaterialAsPayer_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayer)",
"CREATE NONCLUSTERED INDEX iProfitByMaterialAsPayee_Supplier ON [dbo].[Supplier] (ProfitByMaterialAsPayee)",
"CREATE NONCLUSTERED INDEX iProfitByMaterial_Supplier ON [dbo].[PaymentCategory] (ProfitByMaterial)",
"CREATE NONCLUSTERED INDEX iName_Supplier ON [dbo].[PaymentCategory] (Name)",
"CREATE NONCLUSTERED INDEX iCostByMaterial_Supplier ON [dbo].[PaymentCategory] (CostByMaterial)",
"CREATE NONCLUSTERED INDEX iNotInPaymentParticipantProfit_Supplier ON [dbo].[PaymentCategory] (NotInPaymentParticipantProfit)"]

In [14]:
# drop_index_queries = [
# "DROP INDEX iName_AccountType ON [dbo].[AccountType]",
# "DROP INDEX iProfitByMaterialAsPayer_Supplier ON [dbo].[Supplier]",
# "DROP INDEX iProfitByMaterialAsPayee_Supplier ON [dbo].[Supplier]",
# "DROP INDEX iProfitByMaterial_Supplier ON [dbo].[PaymentCategory]",
# "DROP INDEX iName_Supplier ON [dbo].[PaymentCategory]",
# "DROP INDEX iCostByMaterial_Supplier ON [dbo].[PaymentCategory]",
# "DROP INDEX iNotInPaymentParticipantProfit_Supplier ON [dbo].[PaymentCategory]"]

In [15]:
# for query in drop_index_queries:
#     adapter.execute(query)
#     adapter.commit()

In [16]:
for query in create_index_queries:
    adapter.execute(query)
    adapter.commit()

In [17]:
payments = [100, 1000, 10000]
for n in payments:
    adapter.clean_tables(tables=['Payment'])
    start_time = perf_counter()
    adapter.generate_payments(n)
    total_time = perf_counter() - start_time
    print(f'Number of inserted payments: {n}, total time: {total_time} seconds')

Number of inserted payments: 100, total time: 0.6447747469937894 seconds
Number of inserted payments: 1000, total time: 9.79454587699729 seconds
Number of inserted payments: 10000, total time: 664.2468722580088 seconds
